# C/C++ DLL, Unit Test for C/C++, Excel VBA, Python

## 1 C/C++ __stdcall DLL 

  For `Visual Basic` applications (or applications in other languages such as Pascal or Fortran) to call functions in a C/C++ DLL, the functions must be exported using the correct calling convention without any name decoration done by the compiler.
  
VBA can **only** call `__stdcall` functions, not `__cdecl` functions. 
  
*  `__stdcall` creates the correct calling convention for the function (the called function cleans up the stack and parameters are passed from right to left)

* ` __declspec(dllexport)` is used on an `exported` function in a DLL

Below is the example of techniques which facilitate the use of use of MinGW to create DLLs, exporting functions which may be called from Visual Basic Application with Excel. 

Example: Equations for Region4 of [IAPWS-IF97](http://www.iapws.org/relguide/IF97-Rev.pdf)

* 8 Equations for Region 4 

  * 8.1 The Saturation-Pressure Equation (Basic Equation) P33,Eq30
  
  * 8.2 The Saturation-Temperature Equation (Backward Equation) P35, Eq31


### 1.1 Create your DLL with `__stdcall` calling convention

Create a DLL with the following code:

* region4.h

* region4.c

For Windows,export all functions as `__stdcall`.

```c
#define DLLPORT __declspec(dllexport) __stdcall 
```
**NOTE**： The following header `region4.h`  declares the interface for 

* building the **Windows/Linux shared library  

* building an executable that uses the  shared library.


In [1]:
%%file ./demo/src/region4.h

#pragma once

#ifdef __cplusplus
extern "C" {
#endif

#ifdef BUILD_DLL

    #ifdef WIN32
        #define DLLPORT __declspec(dllexport) __stdcall 
    #else
        #define DLLPORT 
    #endif    

#else

    #ifdef WIN32
        #define DLLPORT __declspec(dllimport) __stdcall   
    #else
        #define DLLPORT 
    #endif    

#endif

DLLPORT  double pSat(double T);
DLLPORT  double TSat(double p);
        
#ifdef __cplusplus
	}
#endif        

Overwriting ./demo/src/region4.h


When you create header files for your DLLs, use

* ` __declspec(dllexport) ` adds the `export` directive to the object fileworks

* ` __declspec(dllimport)`  on the declarations of the public symbols

In [3]:
%%file ./demo/src/region4.c
#include <math.h>
#include "region4.h"

//
//  Initialize coefficients for region 4
//
static double n[11] = {0, 0.11670521452767E+04, -0.72421316703206E+06, -0.17073846940092E+02,
                          0.12020824702470E+05, -0.32325550322333E+07, 0.14915108613530E+02,
                         -0.48232657361591E+04, 0.40511340542057E+06, -0.23855557567849E+00,
                          0.65017534844798E+03};

double pSat(double T)
// saturation pressure of water
// pSat in MPa
// T :temperaturein K
//
// pSat = -1: temperature outside range
//
{
    double pS;
    if (T < 273.15 || T > 647.096) // tc_water=647.096
        pS = -1.0;
    else
    {
        double del = T + n[9] / (T - n[10]);
        double aco = del * (del + n[1]) + n[2];
        double bco = del * (n[3] * del + n[4]) + n[5];
        double cco = del * (n[6] * del + n[7]) + n[8];
        pS = pow(2 * cco / (-bco + sqrt(bco * bco - 4 * aco * cco)), 4);
    }
    return pS;
}

double TSat(double p)
// saturation temperature of water
// TSat in K
// p :pressure MPa
//
// TSat=-1: pressure outside range
//
{
    double TS;
    if (p < 0.000611212677 || p > 22.064)
        TS = -1.0;
    else
    {
        double bet = pow(p, 0.25);
        double eco = bet * (bet + n[3]) + n[6];
        double fco = bet * (n[1] * bet + n[4]) + n[7];
        double gco = bet * (n[2] * bet + n[5]) + n[8];
        double dco = 2.0 * gco / (-fco - sqrt(fco * fco - 4.0 * eco * gco));
        TS = 0.5 * (n[10] + dco - sqrt((n[10] + dco) * (n[10] + dco) - 4.0 * (n[9] + n[10] * dco)));
    }
    return TS;
}

Overwriting ./demo/src/region4.c


In [4]:
!gcc -c -DBUILD_DLL -o ./demo/obj/region4.o ./demo/src/region4.c -I./demo/src
!gcc -shared -o ./demo/bin/libregion4.dll ./demo/obj/region4.o  -Wl,--add-stdcall-alias

* -DBUILD_DLL:
   
  * -Dname: `Predefine name as a macro`, with definition 
  

*  -Wl,option 

   Pass **option** as an option to the **linker**. If option contains commas, it is split into multiple options at the commas.


* --add-stdcall-alias:
   
   This adds an undecorated alias for the `exported function names` that is simply **the name of the function** 

### 1.2  Add -static-libgcc ,-output-def=libregion4.def

 Links the GNU `libgcc` library `statically`

In [6]:
!gcc -c -DBUILD_DLL -o ./demo/obj/region4.o ./demo/src/region4.c -I./demo/src
!gcc -shared -o ./demo/bin/libregion4.dll -static-libgcc ./demo/obj/region4.o -Wl,--add-stdcall-alias,-output-def=./demo/bin/libregion4.def

* -static-libgcc

   This option links the GNU `libgcc` library **statically** 
   

* -output-def=libregion4.def

    Name of `.def` file to be created.
    
    **def:** A module-definition  file is a text file containing one or more module statements that describe various attributes of a DLL

In [ ]:
%load ./demo/bin/libregion4.def

In [8]:
%%file ./demo/makefile-region4

CC=gcc
CFLAGS=-DBUILD_DLL

SRCDIR= ./demo/src/
OBJDIR= ./demo/obj/
BINDIR= ./demo/bin/

all: libregion4.dll

libregion4.dll: obj
	 $(CC) -shared -o $(BINDIR)libregion4.dll -static-libgcc $(OBJDIR)region4.o -Wl,--add-stdcall-alias,-output-def=libregion4.def
	 del .\demo\obj\region4.o
    
obj: 
	 $(CC) -c $(CFLAGS) -o $(OBJDIR)region4.o $(SRCDIR)region4.c
     
clean:
	 del .\demo\bin\libregion4.dll

Writing ./demo/makefile-region4


In [9]:
!make -f ./demo/makefile-region4

gcc -c -DBUILD_DLL -o ./demo/obj/region4.o ./demo/src/region4.c
gcc -shared -o ./demo/bin/libregion4.dll -static-libgcc ./demo/obj/region4.o -Wl,--add-stdcall-alias,-output-def=libregion4.def
del .\demo\obj\region4.o


### 1.3 Call Dll from C/C++

In [5]:
%%file ./demo/src/mainReg4.c

#include <stdio.h> 
#include "region4.h"

int main() {
     double T=300.0;
     printf("Saturation P is %f\n", pSat(T));  
     return 0;
}

Overwriting ./demo/src/mainReg4.c


In [6]:
!gcc -c -o ./demo/obj/mainReg4.o ./demo/src/mainReg4.c 
!gcc -o  ./demo/bin/mainReg4 ./demo/obj/mainReg4.o -I./demo/src/ -L./demo/bin/ -lregion4

In [7]:
!.\demo\bin\mainReg4

Saturation P is 0.003537


## 2 Unit Test for C/C++

Unit testing frameworks for C/C++ are most often **third-party products** that are `not distributed as part of the compiler suite`. They help simplify the process of unit testing,

* Microsoft Unit Testing Framework for C++

* [Google Test](https://github.com/abseil/googletest/blob/master/googletest/docs/primer.md)

* Boost.Test

* CTest

* [ThrowTheSwitch Unity：Unit Testing for C](https://github.com/ThrowTheSwitch/Unity)

### 2.1 ThrowTheSwitch Unity：Unit Testing for C

The Simple Unit Testing of Region4 by ThrowTheSwitch Unity

![Unity](./img/unity.jpg)

In [ ]:
%%file ./demo/src/test4_unity.c

#include "region4.h"
#include "unity.h"

//  T,p
static const double tab35[3][2] = {
    {300, 0.00353658941},
    {500, 2.63889776},
    {600, 12.3443146}};

// p,T
static const double tab36[3][2] = {
    {0.1, 372.755919},
    {1, 453.035632},
    {10, 584.149488}};

void setUp(void) {}

void tearDown(void) {}


void test_SaturationP(void)
{
  //  Saturation P line
  for (int i = 0; i < 3; i++)
  {
    double T = tab35[i][0];
    TEST_ASSERT_EQUAL_FLOAT(tab35[i][1], pSat(T));
  }
}

void test_SaturationT(void)
{
  //  Saturation T line
  for (int i = 0; i < 3; i++)
  {
    double p = tab36[i][0];
    TEST_ASSERT_EQUAL_FLOAT(tab36[i][1], TSat(p));
  }
}
 
int main(void)
{
  UNITY_BEGIN();
  RUN_TEST(test_SaturationP);
  RUN_TEST(test_SaturationT);
  return UNITY_END();
}

In [10]:
!gcc -c -o ./demo/obj/unity.o  ./demo/Unity/unity.c -I./demo/Unity/
!gcc -c -o ./demo/obj/test4_unity.o ./demo/src/test4_unity.c -I./demo/src/ -I./demo/Unity/
!gcc -o  ./demo/bin/test4_unity.exe ./demo/obj/test4_unity.o ./demo/obj/unity.o -L./demo/bin/ -lregion4

In [11]:
!.\demo\bin\test4_unity.exe

./demo/src/test4_unity.c:45:test_SaturationP:PASS
./demo/src/test4_unity.c:46:test_SaturationT:PASS

-----------------------
2 Tests 0 Failures 0 Ignored 
OK


### 2.2 Googletest

The Simple Unit Testing of Region4 by  Googletest

https://github.com/google/googletest/blob/master/googletest/docs/primer.md
    
googletest helps you write better C++ tests.

googletest is a testing framework developed by the Testing Technology team with Google's specific requirements and constraints in mind. No matter whether you work on Linux, Windows, or a Mac, if you write C++ code, googletest can help you. And it supports any kind of tests, not just unit tests.    

#### 2.2.1 To create a test

Use the **TEST()** macro to define and name a test function, These are ordinary C++ functions that don't return a value.

* In this function,use the various **googletest assertions** to check values.

* The test's result is determined by the **assertions**

if any assertion in the test fails (either fatally or non-fatally), or if the test crashes, the entire test fails. Otherwise, it succeeds.

```cpp
TEST(TestCaseName, TestName) {
  ... test body ...
}
```

**TEST()** arguments go from general to specific. 

* The first argument is **the name of the test case** 

* the second argument is the **test's name within the test case**

Both names must be valid C++ identifiers, and they should not contain underscore (_). A test's full name consists of its containing test case and its individual name. Tests from different test cases can have the same individual name.


#### 2.2.2  Invoking the Tests

**TEST()** implicitly **register** their tests with googletest. So, unlike with many other C++ testing frameworks, you don't have to re-list all your defined tests in order to run them.

After defining your tests, you can run them with **RUN_ALL_TESTS()** , which returns 0 if all the tests are successful, or 1 otherwise. 

Note：`RUN_ALL_TESTS()` runs **all tests** in your link unit -- they can be from different test cases, or even different source files.

The `::testing::InitGoogleTest()` function parses the command line for googletest flags, and removes all recognized flags. This allows the user to control a test program's behavior via various flags, which we'll cover in [AdvancedGuide](https://github.com/google/googletest/blob/master/googletest/docs/advanced.md). You must call this function before calling `RUN_ALL_TESTS()`, or the flags won't be properly initialized.

In [12]:
%%file ./demo/src/test4_googletest.cpp

#include "region4.h"
#include <gtest/gtest.h>

//  T,p
double tab35[3][2] = {{300, 0.00353658941},
                      {500, 2.63889776},
                      {600, 12.3443146}};
// p,T
double tab36[3][2] = {{0.1, 372.755919},
                      {1, 453.035632},
                      {10, 584.149488}};

// Test Region4:  Saturation P
TEST(Region4Test, SaturationPTest)
{
  for (int i = 0; i < 3; i++)
  {
    double T = tab35[i][0];
    EXPECT_NEAR(tab35[i][1], pSat(T), 1.0e-6);
  }
}

// Test Region4: Saturation T
TEST(Region4Test, SaturationTTest)
{
  for (int i = 0; i < 3; i++)
  {
    double p = tab36[i][0];
    EXPECT_NEAR(tab36[i][1], TSat(p), 1.0e-6);
  }
}

int main(int argc, char **argv)
{
  ::testing::InitGoogleTest(&argc, argv);
  return RUN_ALL_TESTS();
}

Overwriting ./demo/src/test4_googletest.cpp


In [13]:
!g++ -o  ./demo/bin/test4_googletest.exe ./demo/src/test4_googletest.cpp -L./demo/bin/  -lregion4 -lgtest

In [14]:
!.\demo\bin\test4_googletest.exe

[==========] Running 2 tests from 1 test suite.
[----------] Global test environment set-up.
[----------] 2 tests from Region4Test
[ RUN      ] Region4Test.SaturationPTest
[       OK ] Region4Test.SaturationPTest (0 ms)
[ RUN      ] Region4Test.SaturationTTest
[       OK ] Region4Test.SaturationTTest (0 ms)
[----------] 2 tests from Region4Test (0 ms total)

[----------] Global test environment tear-down
[==========] 2 tests from 1 test suite ran. (1 ms total)
[  PASSED  ] 2 tests.


#### 2.2.3  Invoking with gtest_main

But maybe you think that writing all those **main()** functions is too much work? 

We agree with you completely and that's why Google Test provides a basic implementation of **main()** 

If it fits your needs, then just **link** your test with **gtest_main** library and you are good to go

```bash
 -lgtest_main
```

```bash
!g++ -o  ./demo/bin/test4_gtest.exe ./demo/src/test4_gtest_main.cpp -L./demo/bin/  -lregion4 -lgtest -lgtest_main
```

In [ ]:
%%file ./demo/src/test4_gtest_main.cpp

#include "region4.h"
#include <gtest/gtest.h>

//  T,p
double tab35[3][2] = {{300, 0.00353658941},
                      {500, 2.63889776},
                      {600, 12.3443146}};
// p,T
double tab36[3][2] = {{0.1, 372.755919},
                      {1, 453.035632},
                      {10, 584.149488}};

// Test Region4:  Saturation P
TEST(Region4Test, SaturationPTest)
{
  for (int i = 0; i < 3; i++)
  {
    double T = tab35[i][0];
    EXPECT_NEAR(tab35[i][1], pSat(T), 1.0e-6);
  }
}

// Test Region4: Saturation T
TEST(Region4Test, SaturationTTest)
{
  for (int i = 0; i < 3; i++)
  {
    double p = tab36[i][0];
    EXPECT_NEAR(tab36[i][1], TSat(p), 1.0e-6);
  }
}

In [ ]:
!g++ -o  ./demo/bin/test4_gtest.exe ./demo/src/test4_gtest_main.cpp -L./demo/bin/  -lregion4 -lgtest -lgtest_main

In [ ]:
!.\demo\bin\test4_gtest.exe

## 3 Call DLL from Excel VBA(64bits)

Do as the following steps:

**1** `libregion4.dll` in the default path of windows'dll `C:\windows\system`


**2** `demo-r4.xlsm` in `./demo/src/`


**3**  create the `VBA modules` to call the DLL.

Into **VBA**, then， create the module **mathfuns** to call library:

```vba
Declare PtrSafe Function pSat Lib "libregion4" (ByVal T As Double) As Double
Declare PtrSafe Function TSat Lib "libregion4" (ByVal p As Double) As Double

Public Function CalpSat(ByVal T As Double) As Double
    CalpSat = region4.pSat(T)
End Function
 
Public Function  CalTSat(ByVal p As Double) As Double
    CalTSat = region4.TSat(p)
End Function
```
**4**  call VBA methods in cells

![demo-r4](./img/demo-r4.jpg)

## 4 call_stdcall DLL from Python

```python
 windll.LoadLibrary
```

In [ ]:
%%file ./demo/src/region4.py

from ctypes import windll,c_double

flib = windll.LoadLibrary('./demo/bin/libregion4.dll')

def pSat(T):
    flib.pSat.argtypes = [c_double]
    flib.pSat.restype  = c_double
    return flib.pSat(T)

def TSat(p):
    flib.TSat.argtypes = [c_double]
    flib.TSat.restype  = c_double
    return flib.TSat(p)

**add `mathfuns.py` into the interperte search path**

In [ ]:
import sys
sys.path.append('./demo/src')

In [ ]:
%%file ./demo/src/test4.py

import unittest
from region4 import *

class Region4Test (unittest.TestCase):

    def setUp(self):
        # IF97-dev,Table35 Page 34 : T(K) p(MPa)
        self.tab35=[[300, 0.353658941e-2],
                    [500, 0.263889776e1],
                    [600, 0.123443146e2]]

        # IF97-dev, Table 36 Page 36 :  p(MPa) T(K)
        self.tab36=[[0.1, 0.372755919e3],
                    [  1, 0.453035632e3],
                    [ 10, 0.584149488e3]]

    def test_pSat(self):
        places = 6
        for item in  self.tab35:
             self.assertAlmostEqual(pSat(item[0]),item[1],places)

    def test_TSat(self):
        places = 6
        for item in  self.tab36:
             self.assertAlmostEqual(TSat(item[0]),item[1],places)
     

if __name__ == '__main__':
    unittest.main()       

In [ ]:
%run ./demo/src/test4.py

## Reference

###  C/C++

* ctypes http://docs.python.org/3/library/ctypes.html

* Unit Test for C/C++

    * Simple Unit Testing for C：https://github.com/ThrowTheSwitch/Unity

    * Write unit tests for C/C++ in Visual Studio https://docs.microsoft.com/en-us/visualstudio/test/writing-unit-tests-for-c-cpp?view=vs-2017'

    * Googletest - Google Testing and Mocking Framework https://github.com/google/googletest

    * Arpan Sen. [A quick introduction to the Google C++ Testing Framework](https://www.ibm.com/developerworks/aix/library/au-googletestingframework.html)
    
    
###  Microsoft Excel

* Excel VBA Programming:  http://www.homeandlearn.org/the_excel_vba_editor.html

* [Language reference for Visual Basic for Applications(VBA)](https://docs.microsoft.com/en-us/office/vba/api/overview/language-reference)

* [Excel VBA reference](https://docs.microsoft.com/en-us/office/vba/api/overview/excel)

* [Excel add-in tutorial](https://docs.microsoft.com/en-us/office/dev/add-ins/tutorials/excel-tutorial)

* DLLs in Visual C++ https://msdn.microsoft.com/en-us/library/1ez7dh12.aspx

* Calling DLL Functions from Visual Basic Applications https://msdn.microsoft.com/en-us/library/dt232c9t.aspx
   